<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<p style="padding: 5px;
              color:white;">
                   <center> Source #00: <a href="https://www.youtube.com/watch?v=sa-TUpSx1JA&ab_channel=CoreySchafer" style="color:gold" > The one I wish I had started with(corey schafer)</a> </center>               
                   <center> Source #1: <a href="https://www.kaggle.com/ammarnassanalhajali/brain-tumor-3d-tr" style="color:gold" >ammarnassanalhajali nb</a> </center>  
                   <center> Source #2: <a href="https://www.youtube.com/watch?v=UQQsYXa1EHs&ab_channel=Kite" style="color:gold" > Kite</a> </center>   
                   <center> Source #2: <a href="https://docs.python.org/3/library/re.html" style="color:gold" > The docs</a> </center> 
                   <center> Source #2: <a href="https://cheatography.com/mutanclan/cheat-sheets/python-regular-expression-regex/" style="color:gold" > Regex cheatsheet</a> </center>
</p>               
</div>

<center><h1>In this Notebook we will be going through and learning the basics of Regular Expressions(RE or Regex) and then applying them to our dataset</center>

### The Goal: provide a helpful tool for your preprocessing toolset and help me build my own skill set via presentation

#### One of the applied regular expressions we will work with and that was inspiration for this post is from @ammarnassanalhajali in his Original Notebook: <a href="https://www.kaggle.com/ammarnassanalhajali/brain-tumor-3d-training" style="color:Navy" >Notebook</a>

<center><h2>imports</h1></center>

In [ ]:
import os
#for file management
import glob
#for precise file selection(low verbosity)
import time
#return the time in seconds since the epoch as a floating point number. A good way to watch your training progress quantitatively
import regex as re
#import regular expression matching operations. alows for facilitated use of string comparison. Docs:https://docs.python.org/3/library/re.html
import math
#provides mathematical functions established within the c-standard. gives ceil, factorial, etc. Docs:https://docs.python.org/3/library/math.html
import numpy as np
#linear algebra
import pandas as pd
#succinct array and data handling


<center><h2>Context</h2></center>
<br>
<center><h3>Regex can be used in order to scrape a webpage, review your code more quickly, parse files in a loop, check user input, and the list goes on. However, it is a library so there are some things we need to know going forward<br>in order to maximize our time working with them. The rest we should(as always) supplement by rereading the documentation when we are profoundly lost(which can happen at first...and in general) </h3></center>

<center><h2>How to: Lets talk special characters, general knowledge, and give an example for reading email notation</h1></center>

> taking a peek at special characters

In [ ]:
# special characters, a few to keep in mind as we continue(see the regex cheat sheet I linked for far more):

# .    the default mode that will catch all characters except for new line

# ^    matches the start of the string

# $    matches the end of the string just before the newline at the end of the string

# *    matches the preceding RE to as many repetitions of it as there are 

# []   match a set of characters

# \    escape special characters

# | ex.  RE1 | RE2: Match either RE1 or RE2

# \Z   match only at the end of a string

# re.findall() return all non-overlapping matches of pattern in string as list of strings

# ?    match 0 or 1 repetitions of RE

#working with ranges concepts:

#[0-5][0-9] will match all the two-digits numbers from 00 to 59

#see docs for the special characters list

#non special chracters that used to be special( or ones you may have assumed were special)
#'!', '"', '%', "'", ',', '/', ':', ';', '<', '=', '>', '@', and "`" are no longer escaped.

> some interesting information. turns out regex plus worried engineers can = a boggled reader. but now we know to watch out for it :)

one last interesting note on escaped characters and special characters: <br>
" you'll typically see everything escaped (e.g. [a-z\%\$\#\@\!\-\_]) simply because the engineer doesn't know what's "special" and what's not... so they "play it safe" and obfuscate the expression with loads of excessive backslashes." <br>
-[source](https://stackoverflow.com/questions/9589074/regex-should-hyphens-be-escaped)

> reading an email from user input

In [ ]:
#lets take user input and return whether or not they inputted an email

email_pattern = "[a-zA-Z0-9]+@[a-zA-Z]+\.(com|edu|net)"
# [a-zA-Z0-9] here we are looking to match a set of characters that can be read as follows:
'''
starting from the beginning of the string match if the string has...
any characters in the lower case alphabet or a-z
any chracters in the upper case alphabet or A-Z
and any digits between 0-9
thus a-zA-Z0-9 where the hypen(-) indicates a range 
'''
# +@[a-zA-Z] the plus indicates we break/stop when we reach the subsequent value in this case the @ symbol

#+\.(com|edu|net)
'''
so now we have collected a string and an @ symbol and from there we continue matching with the previous logic until we meet +\. which means:
+ stop previous matching at a period(.) the backslash is here because a period is a special character 

(com|edu|net) where the () indicates that the contents matched inside are the last ones and the | indicates or
from here we match the next section if it includes com or edu or net and ends the matching function
'''

> we set our pattern now lets run our logic (I will leave the output after i typed the crazy@floridaman.net email when prompted in initial run)

In [ ]:
user_input = input()
if(re.search(email_pattern, user_input)):
#we run a conditional that uses the search function from the re module that
#takes the parameter that is your re patter and casts it to your input and 
#returns what I assume is an object that has a few attributes, the one that
#we care about here being True or False as to if we have an email present in the parsed input

    print("email")
else:
    print("non-email")

<center><h2>Applied: basic file handling of rsna comp dir</h1></center>

<center><h2>Heads Up! try ctrl-f and then enter in some regular expressions in order to find digits or another example.<br> its a great time saver and allows you to test your implementation before coding it(note that you need to press<br> the 3 dots in the drop down and then click on the .* in the corner of the menu</h2></center>

> lets look at our main directory, choose an example subfile and define the pattern it has in regex notation

In [ ]:
#stepping up the complexity a bit here. skip to the next example for the sorted function used in a notebook for efficiently loading 3d dicom images

'''

Lets create the initial_pattern for the data within our main dir and then change it to remove the hyphen in the usual ../"Image-1" subfile

'''

#re.sub(initial_pattern, new_pattern, file_path)
dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'

p1img1 = 'train/00000/FLAIR/Image-1.dcm' #p1img1 for patient one image one

> now that we have a target path, we need an appropriate regular expression match in order to modify the contents of the pattern to remove the hyphen later

In [ ]:
#Solution
initial_pattern = r"(\w+/\w+/\w+/[a-zA-Z0-9]+)(\-)(\d\.dcm)"
#we set the initial_pattern variable to a raw string whose contents we make 3 groups, 1 and 3 separate from 2 that contains the hyphen 
#\w indicates any character
#\w+ indicates any word character of an entire string until you stop it from matching anymore at a different, non word string value
    # w+ == [a-zA-Z0-9] , i just use both here because i initially was afraid of doing a w++ while trying to end the w+ statement
#\- indicates the hypen
# () indicates a grouping and it is excluded from what is read as the pattern semantically
#\d indicates one digit between 0-9 
#\. indicates a period
#dcm is just the matching to the letters dcm spelled out
# in effect we simply concatenate the matched values in order with a forward slash(/) when indicated and have also made sure to separate the hyphen so we can remove it in the next code block

> we can quickly run a test with this script and use some conditionals to check and change our path value

In [ ]:
test_path = 'train/00000/FLAIR/Image-1.dcm'
if(re.search(initial_pattern, test_path )):
    #the search function compares the initial_pattern and the path and determines if they have the same regex representation
    print("works")
    new = re.sub(initial_pattern, r'\1\3', test_path) #exclude grouping 2, \2 that holds the hyphen
    #the re.sub() method substitutes the values of our initial pattern out using the middle parameter which selects which groups we would like to read from our test_path that is casted with the initial_pattern
    print(new)
else: #as usual, regex involves some guess and check so a conditional with some level of feedback is very helpful
    print("nah")
#     initial_pattern = re.compile(initial_pattern)
    print(initial_pattern)
    #keep

> and as we had guessed earlier there is indeed an object that logs the info about an re

In [ ]:
# initial_pattern = re.compile(initial_pattern)
print(re.search(initial_pattern, test_path))

<center><h2>Applied: Sorted function for </h1></center>

In [ ]:
#for now we will ignore the context and just plug in some params so we can test the logic

scan_id = '00001'
data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-1.dcm' 

def load_dicom_images_3d(scan_id, num_imgs=64, img_size=256, mri_type="FLAIR", split="train", rotate=0):

       files = sorted(glob.glob(f"{data_directory}/{split}/{scan_id}/{mri_type}/*.dcm"), 
                  #we set files equal to a sorted function
                      #glob.glob function parses an f string using our input parameters
                      
               key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
                  #we set our key value that will dictate our sorting to a lambda function that utilizes an regexpression
                        #logic annotated in expanded form example below
    
    #lambda function in expanded form
    '''
                        --define lambda function--
    def lambda(var):
                        ---for x in the contents of the 1d array generated by parsing each individual character grouping via the regex expression and re.findall expression---
        for x in re.findall((r'[^0-9]|[0-9]+', var)):
                        ---if x is a digit, return the value as an integer in an array and using the sorted value use that mapped array that was generated at the given-----
                        ---values to return a files variable the is in numerical ascending order based on Image-#--------
                if x.isdigit():
                    return int(x)
                else:
                    return x
                
    '''               

> what the re.findall is doing in the background

In [ ]:
file = sorted(glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-1.dcm'), 
          #we set files equal to a sorted function
              #glob.glob function parses an f string using our input parameters

       key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
print(file)

In [ ]:
pattern = r'[^0-9]|[0-9]+'
re.findall(pattern, '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-1.dcm' )

<center><h2>Applied Bonus=>identify files from within directories and access them via a loop  </h1></center>

> source: https://stackoverflow.com/questions/39293968/how-do-i-search-directories-and-find-files-that-match-regex

In [ ]:
import os
#importing os to enable ability to change path values from a script
import re
#importing regular expressions (import regex as re can help prevent errors at times but we'll leave this import as it was from its post)

regex = re.compile('(.*zip$)|(.*rar$)|(.*r01$)')
#create a regex object using the re.compile function that is a grouping of either a match to a zip file, rar file, or ro1 file
    # .* effectively grabs all the characters of a raw string with the . and then for as many iterations that there are but is then capped at the end with the word zip as it is connected to a $ end of string indicator
                #this works so well because we know nothing comes after the file type in a file name
rx = '(.*zip$)|(.*rar$)|(.*r01$)'
#set a string to be 3 possible groupings

for root, dirs, files in os.walk("../Documents"):
    #traverse through the builtin root dirs and files of frame of the os library with the walk function at the given folder 
  for file in files:
    #for files in files use the re.match regex function to check if at any group 1,2,or3 that we setup in rx = 1|2|3 we return
    #a print statement declaring what file type it is. from there you can make functionalities based on the file type
    res = re.match(rx, file)
    if res:
      if res.group(1):
        print("ZIP",file)
      if res.group(2):
        print("RAR",file)
      if res.group(3):
        print("R01",file)

for those who made it to here, here is a funny stackoverflow post that I found on my quest to comprehend regex: https://stackoverflow.com/questions/1454913/regular-expression-to-find-a-string-included-between-two-characters-while-exclud

"Easy done"-Cletus 

hop in our ml discord! 

@ https://discord.gg/E5hJkwNT